# Generalizing mutagenesis pipeline

Allowing for facile input of a transcript id/other information to automate process.

In [1]:
import numpy as np
import gffutils
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd 
from pegg import prime
import Bio.Seq

/Users/samgould/opt/anaconda3/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
#importing human reference genome
filepath = '/Users/samgould/Desktop/FSR Lab/reference files/GRCm39 (mouse)/GCF_000001635.27_GRCm39_genomic.fna.gz'
chrom_dict, i = prime.genome_loader(filepath)

In [3]:
#GRCh38 database
#loading in genome annotations
db = gffutils.FeatureDB('/Users/samgould/Desktop/FSR Lab/reference files/gencode_vm33_GRCm39.db', keep_order=True)

#accessed genome annotations using gffutils package
#canonical MANE transcript for CDK9 (CDK9-201)
tx = 'ENST00000373264.5'


def tx_processor(tx):
    cds = list(db.children(tx, order_by='+end', featuretype=['CDS']))
    start_end_cds = [[i.start, i.end] for i in cds]
    strand = db[tx].strand
    chrom = db[tx].chrom
    #print(chrom[3:])
    #print(strand)
    #print(db[tx].attributes)

    #including 20 nt buffer on either side of exon for generating PAM sequences
    start_end_cds_20 = []
    buffer = 20
    for k in start_end_cds:
        h = []
        for idx, j in enumerate(k):
            if idx==0:
                h.append(j-buffer)
            if idx==1:
                h.append(j+buffer)
        start_end_cds_20.append(h)

    return start_end_cds_20, start_end_cds, chrom, strand

#tx_processor(tx)

In [4]:
def mutagenize(tx, chrom_dict):
    
    start_end_cds_20, start_end_cds, chrom, strand = tx_processor(tx)
    #chr9 = chrom_dict[9]
    if chrom != 'chrX':
        chr9 = chrom_dict[int(chrom[3:])]
    else:
        chr9 = chrom_dict[chrom[3:]]
    #chr9 = chrom_dict[int(chrom[3:])]

    plus_count = 0
    minus_count = 0

    plus_pam_loc_start = []
    plus_pam_loc_end = []
    PAM_plus = []
    plus_protospacer = []
    plus_proto_start = []
    plus_proto_end = []
    plus_ideal_start = []
    plus_ideal_end = []
    plus_ideal_window = []
    plus_exon_list = []

    minus_pam_loc_start = []
    minus_pam_loc_end = []
    PAM_minus = []
    minus_protospacer = []
    minus_proto_start = []
    minus_proto_end = []
    minus_ideal_start = []
    minus_ideal_end = []
    minus_ideal_window = []
    minus_exon_list = []

    #need to modify this to run it more generally!!!
    #COME BACK TO THIS
    #exon_list = [1,2,3,4,5,6,7]
    if strand=='+':
        exon_list = list(range(1, len(start_end_cds)+1))
    elif strand=='-':
        exon_list = list(range(1, len(start_end_cds)+1))[::-1]

    for idx1, i in enumerate(start_end_cds_20):
        s = i[0]-1
        e = i[1]

        sub_chrom=str(chr9[s:e]).upper()
        #print(sub_chrom)
        loc_list = list(range(s, e))

        for idx, k in enumerate(sub_chrom):
            #first deal with the plus strand
            if k=='G':
                
                if idx-20 >=0:
                    plus_count+=1
                    plus_pam_loc_start.append(loc_list[idx])
                    plus_pam_loc_end.append(loc_list[idx]+1)

                    PAM_plus.append(str(chr9[loc_list[idx]-1:loc_list[idx]+1]).upper())
                    
                    proto_start = loc_list[idx]-21
                    proto_end = loc_list[idx]-1
                    proto = str(chr9[proto_start: proto_end]).upper()
                    plus_protospacer.append(proto)
                    plus_proto_start.append(proto_start+1)
                    plus_proto_end.append(proto_end)

                    #and ideal editing window
                    plus_ideal_start.append(proto_start+3+1)
                    plus_ideal_end.append(proto_start+8)
                    plus_ideal_window.append(str(chr9[proto_start+3: proto_start+8]).upper())
                    plus_exon_list.append(exon_list[idx1])

            #then the minus strand
            if k=='C':

                if idx+20<=len(sub_chrom):
                    minus_count+=1

                    minus_pam_loc_end.append(loc_list[idx])
                    minus_pam_loc_start.append(loc_list[idx]+1)

                    PAM_minus.append(str(chr9[loc_list[idx]:loc_list[idx]+2].reverse_complement()).upper())

                    proto_end = loc_list[idx]+2
                    proto_start = loc_list[idx] + 22
                    proto = str(chr9[proto_end: proto_start].reverse_complement()).upper()

                    minus_protospacer.append(proto)
                    minus_proto_start.append(proto_start)
                    minus_proto_end.append(proto_end+1)

                    #and ideal editing window
                    minus_ideal_start.append(proto_start-3)
                    minus_ideal_end.append(proto_start-8+1)
                    minus_ideal_window.append(str(chr9[proto_start-8: proto_start-3].reverse_complement()).upper())
                    minus_exon_list.append(exon_list[idx1])

    col_names = ['protospacer', 'exon', 'proto_start', 'proto_end', 'PAM', 'PAM_start', 'PAM_end', 'ideal_start', 'ideal_end', 'ideal_window']

    cols = [plus_protospacer, plus_exon_list, plus_proto_start,plus_proto_end,PAM_plus,plus_pam_loc_start,plus_pam_loc_end, plus_ideal_start, plus_ideal_end, plus_ideal_window ]
    plus_df = pd.DataFrame(dict(zip(col_names, cols)))
    plus_df['strand'] = '+'


    cols_minus = [minus_protospacer, minus_exon_list, minus_proto_start,minus_proto_end,PAM_minus,minus_pam_loc_start,minus_pam_loc_end, minus_ideal_start, minus_ideal_end, minus_ideal_window ]

    minus_df = pd.DataFrame(dict(zip(col_names, cols_minus)))
    minus_df['strand'] = '-'
    

    #filter out guides whose ideal editing window doesn't hit in an exon
    cds_hit = []
    for i, val in plus_df.iterrows():
        ideal_start = val['ideal_start']
        ideal_end = val['ideal_end']-1
        
        hit=False
        for k in start_end_cds:
            exon_coords = list(range(k[0], k[1]+1))
            if ideal_start in exon_coords:
                hit=True
            if ideal_end in exon_coords:
                hit=True
            
        cds_hit.append(hit)

    plus_df['CDS_hit'] = cds_hit

    #and do the same for the minus strand
    #filter out guides whose ideal editing window doesn't hit in an exon
    cds_hit = []
    for i, val in minus_df.iterrows():
        ideal_start = val['ideal_start']-1
        ideal_end = val['ideal_end']
        
        hit=False
        for k in start_end_cds:
            exon_coords = list(range(k[0], k[1]+1))
            if ideal_start in exon_coords:
                hit=True
            if ideal_end in exon_coords:
                hit=True
            
        cds_hit.append(hit)

    minus_df['CDS_hit'] = cds_hit
    
    return pd.concat((plus_df, minus_df))

# Simulated Mutagenesis 


In [5]:
wt_seq = 'KKGTACYW'
mut_seq = 'KGGTTCRW'

def hgvsp_simple(wt_seq, mut_seq):
    if wt_seq==mut_seq:
        return 'WT'
    else:
        pos_mutated = []

        for i in range(len(wt_seq)):
            if wt_seq[i] != mut_seq[i]:
                pos_mutated.append(i + 1)
        
        hg = ''
        for idx, pos in enumerate(pos_mutated):
            if idx==0:
                hg+= f'{wt_seq[pos-1]}{pos}{mut_seq[pos-1]}'
            else:
                hg+= f'_{wt_seq[pos-1]}{pos}{mut_seq[pos-1]}'
        

        return hg

hgvsp_simple(wt_seq, mut_seq)

'K2G_A5T_Y7R'

In [6]:
import itertools

def ABE_mutater(ideal_window):
    #count the As
    count_A = ideal_window.count('A')

    #generate all possible combinatoric mutation possibilities
    list_combs = []
    for i in range(1, count_A+1):

        list_combs+=list(itertools.combinations(range(count_A), i))

    #index the As
    l_key = []
    counter=0
    for i in ideal_window:
        if i=='A':
            l_key.append(counter)
            counter+=1
        else:
            l_key.append('None')

    #generate the mutated sequences
    mutated_ideal = []
    for i in list_combs:
        len_combination = len(i)

        mut_seq = ''
        #for k in i:
        for idx, base in enumerate(ideal_window):
            if l_key[idx] in i:
                mut_seq+='G'
            else:
                mut_seq+=base

        mutated_ideal.append(mut_seq)

    return mutated_ideal

def CBE_mutater(ideal_window):
    #count the As
    count_C = ideal_window.count('C')

    #generate all possible combinatoric mutation possibilities
    list_combs = []
    for i in range(1, count_C+1):

        list_combs+=list(itertools.combinations(range(count_C), i))

    #index the As
    l_key = []
    counter=0
    for i in ideal_window:
        if i=='C':
            l_key.append(counter)
            counter+=1
        else:
            l_key.append('None')

    #generate the mutated sequences
    mutated_ideal = []
    for i in list_combs:
        len_combination = len(i)

        mut_seq = ''
        #for k in i:
        for idx, base in enumerate(ideal_window):
            if l_key[idx] in i:
                mut_seq+='T'
            else:
                mut_seq+=base

        mutated_ideal.append(mut_seq)

    return mutated_ideal

In [7]:
def plus_mutate(plus_df):

    cds_plus = plus_df[plus_df['CDS_hit']==True]
    cds_plus = cds_plus.reset_index().drop(columns='index')


    cds_plus['gRNA_id'] = [f'gRNA_{i+1}' for i in range(len(cds_plus))]

    #perform combinatoric replacement
    #ABE: A>G
    #CBE: C>T
    ABE_amenable = []
    CBE_amenable = []

    ABE_edits = []
    ABE_edit_guides = []
    CBE_edits = []
    CBE_edit_guides = []

    for i, val in cds_plus.iterrows():
        ideal_window = val['ideal_window']
        gRNA = val['gRNA_id']

        #start with ABE:
        if 'A' in ideal_window:
            ABE_amenable.append(True)
            ABE_mutations = ABE_mutater(ideal_window)
            for k in ABE_mutations:
                ABE_edits.append(k)
                ABE_edit_guides.append(gRNA)

        else:
            ABE_amenable.append(False)

        if 'C' in ideal_window:
            CBE_amenable.append(True)
            CBE_mutations = CBE_mutater(ideal_window)
            for k in CBE_mutations:
                CBE_edits.append(k)
                CBE_edit_guides.append(gRNA)

        else:
            CBE_amenable.append(False)


    cds_plus['ABE_amenable'] = ABE_amenable
    cds_plus['CBE_amenable'] = CBE_amenable

    CBE_edits_df = pd.DataFrame(dict(zip(['gRNA_id', 'edit'], [CBE_edit_guides, CBE_edits])))
    ABE_edits_df = pd.DataFrame(dict(zip(['gRNA_id', 'edit'], [ABE_edit_guides, ABE_edits])))
    CBE_edits_df['Editor'] = 'CBE'
    ABE_edits_df['Editor'] = 'ABE'

    return cds_plus, CBE_edits_df, ABE_edits_df

def minus_mutate(minus_df, plus_df):
    cds_minus = minus_df[minus_df['CDS_hit']==True]
    cds_minus = cds_minus.reset_index().drop(columns='index')


    cds_minus['gRNA_id'] = [f'gRNA_{i+1}' for i in range(len(plus_df), len(plus_df)+len(cds_minus))]

    #perform combinatoric replacement
    #ABE: A>G
    #CBE: C>T
    ABE_amenable = []
    CBE_amenable = []

    ABE_edits = []
    ABE_edit_guides = []
    CBE_edits = []
    CBE_edit_guides = []

    for i, val in cds_minus.iterrows():
        ideal_window = val['ideal_window']
        gRNA = val['gRNA_id']

        #start with ABE:
        if 'A' in ideal_window:
            ABE_amenable.append(True)
            ABE_mutations = ABE_mutater(ideal_window)
            for k in ABE_mutations:
                ABE_edits.append(k)
                ABE_edit_guides.append(gRNA)

        else:
            ABE_amenable.append(False)

        if 'C' in ideal_window:
            CBE_amenable.append(True)
            CBE_mutations = CBE_mutater(ideal_window)
            for k in CBE_mutations:
                CBE_edits.append(k)
                CBE_edit_guides.append(gRNA)

        else:
            CBE_amenable.append(False)


    cds_minus['ABE_amenable'] = ABE_amenable
    cds_minus['CBE_amenable'] = CBE_amenable

    CBE_edits_df = pd.DataFrame(dict(zip(['gRNA_id', 'edit'], [CBE_edit_guides, CBE_edits])))
    ABE_edits_df = pd.DataFrame(dict(zip(['gRNA_id', 'edit'], [ABE_edit_guides, ABE_edits])))
    CBE_edits_df['Editor'] = 'CBE'
    ABE_edits_df['Editor'] = 'ABE'

    return cds_minus, CBE_edits_df, ABE_edits_df

In [8]:
def simulate_mutations_plus(input_df, WT_transcript_full, start_end_cds, strand, chrom_dict, chrom):

    #chr9 = chrom_dict[int(chrom[3:])]
    if chrom != 'chrX':
        chr9 = chrom_dict[int(chrom[3:])]
    else:
        chr9 = chrom_dict[chrom[3:]]

    hgvsp_holder = []
    potential_splice_holder = []

    wt_seq = str(Bio.Seq.Seq(WT_transcript_full).transcribe().translate())

    for i, val in input_df.iterrows():
        wt_ideal_seq = val['ideal_window']
        mut_ideal_seq = val['edit']
        ideal_start = val['ideal_start']
        ideal_end = val['ideal_end']

        mut_transcript = ''
        wt_transcript = ''

        potential_splice = False

        for i in start_end_cds:
            s = i[0]
            e = i[1]
   

            contained=False
            contained_fully = False
            contained_start = False
            contained_end = False

            if ((ideal_end <= e) and (ideal_end>=s) and (ideal_start <= e) and (ideal_start>=s)):
                contained=True
                contained_fully=True

            else:
                if ((ideal_start <= e) and (ideal_start>=s)):
                    contained=True
                    contained_start = True
                    potential_splice=True

                if ((ideal_end <= e) and (ideal_end>=s)):
                    contained=True
                    contained_end=True
                    potential_splice=True
                    
            if contained==False:
                mut_transcript+=chr9[s-1:e]
                wt_transcript+=chr9[s-1:e]
            
            elif contained==True:
                #NEED TO DEAL WITH EDGE CASES WHERE IDEAL EDITING WINDOW OVERLAPS WITH BEGINNING/END OF EXON
                #the first example (first set of parameters) has this precise issue...
                if contained_fully == True:
                    sub_tx_wt = chr9[s-1:ideal_start-1] + wt_ideal_seq + chr9[ideal_end:e]
                    sub_tx_mut = chr9[s-1:ideal_start-1] + mut_ideal_seq + chr9[ideal_end:e]
                    mut_transcript+=sub_tx_mut
                    wt_transcript +=sub_tx_wt

                elif contained_start==True:
                    #here the end needs to be cut off
                    
                    sub_tx_mut = chr9[s-1:ideal_start-1] + mut_ideal_seq[:e-ideal_end]
                    sub_tx_wt = chr9[s-1:ideal_start-1] + wt_ideal_seq[:e-ideal_end]
                    #print(val['gRNA_id'])
                    #print(chr9[s-1:ideal_start])
                    #print(wt_ideal_seq[:e-ideal_end-1])
                    mut_transcript+=sub_tx_mut
                    wt_transcript +=sub_tx_wt


                elif contained_end==True:
                    #here the start needs to be cut off
                    sub_tx_mut = mut_ideal_seq[s-ideal_start:] + chr9[ideal_end:e]
                    sub_tx_wt = wt_ideal_seq[s-ideal_start:] + chr9[ideal_end:e]
                    #print(sub_tx_wt)
                    mut_transcript+=sub_tx_mut
                    wt_transcript +=sub_tx_wt


        if strand=='+':
            #print(sub_tx_wt)
            #print(wt_transcript.upper() + '\n'+ WT_transcript_full.upper())
            
            assert wt_transcript.upper()==WT_transcript_full.upper() , f'non-matching WT transcript | {val["gRNA_id"]} {val["edit"]} | {potential_splice} | {sub_tx_wt}'


            mut_seq = str(Bio.Seq.Seq(mut_transcript).transcribe().translate())

            hgvsp = hgvsp_simple(wt_seq, mut_seq)
            hgvsp_holder.append(hgvsp)
            
            potential_splice_holder.append(potential_splice)
            
        elif strand=='-':

            assert wt_transcript.reverse_complement().upper()==WT_transcript_full.upper(), f'non-matching WT transcript | {val["gRNA_id"]} {val["edit"]} | {potential_splice} | {sub_tx_wt}'


            mut_seq = str(Bio.Seq.Seq(mut_transcript).reverse_complement().transcribe().translate())

            hgvsp = hgvsp_simple(wt_seq, mut_seq)
            hgvsp_holder.append(hgvsp)
            
            potential_splice_holder.append(potential_splice)

    return hgvsp_holder, potential_splice_holder


def simulate_mutations_minus(input_df, WT_transcript_full, start_end_cds, strand, chrom_dict, chrom):

    #chr9 = chrom_dict[int(chrom[3:])]
    if chrom != 'chrX':
        chr9 = chrom_dict[int(chrom[3:])]
    else:
        chr9 = chrom_dict[chrom[3:]]
    
    hgvsp_holder = []
    potential_splice_holder = []

    wt_seq = str(Bio.Seq.Seq(WT_transcript_full).transcribe().translate())

    for i, val in input_df.iterrows():
        wt_ideal_seq = val['ideal_window']
        mut_ideal_seq = val['edit']
        ideal_start = val['ideal_end'] #swapped these
        ideal_end = val['ideal_start']

        wt_ideal_seq = str(Bio.Seq.Seq(wt_ideal_seq).reverse_complement())
        mut_ideal_seq = str(Bio.Seq.Seq(mut_ideal_seq).reverse_complement())

        mut_transcript = ''
        wt_transcript = ''

        potential_splice = False

        for i in start_end_cds:
            s = i[0]
            e = i[1]

            contained=False
            contained_fully = False
            contained_start = False
            contained_end = False

            if ((ideal_end <= e) and (ideal_end>=s) and (ideal_start <= e) and (ideal_start>=s)):
                contained=True
                contained_fully=True

            else:
                if ((ideal_start <= e) and (ideal_start>=s)):
                    contained=True
                    contained_start = True
                    potential_splice=True

                if ((ideal_end <= e) and (ideal_end>=s)):
                    contained=True
                    contained_end=True
                    potential_splice=True
                    
            if contained==False:
                mut_transcript+=chr9[s-1:e]
                wt_transcript+=chr9[s-1:e]
            
            elif contained==True:
                #NEED TO DEAL WITH EDGE CASES WHERE IDEAL EDITING WINDOW OVERLAPS WITH BEGINNING/END OF EXON
                #the first example (first set of parameters) has this precise issue...
                if contained_fully == True:
                    sub_tx_wt = chr9[s-1:ideal_start-1] + wt_ideal_seq + chr9[ideal_end:e]
                    sub_tx_mut = chr9[s-1:ideal_start-1] + mut_ideal_seq + chr9[ideal_end:e]
                    mut_transcript+=sub_tx_mut
                    wt_transcript +=sub_tx_wt

                elif contained_start==True:
                    #here the end needs to be cut off
                    
                    sub_tx_mut = chr9[s-1:ideal_start-1] + mut_ideal_seq[:e-ideal_end]
                    sub_tx_wt = chr9[s-1:ideal_start-1] + wt_ideal_seq[:e-ideal_end]
                    #print(val['gRNA_id'])
                    #print(chr9[s-1:ideal_start])
                    #print(wt_ideal_seq[:e-ideal_end-1])
                    mut_transcript+=sub_tx_mut
                    wt_transcript +=sub_tx_wt


                elif contained_end==True:
                    #here the start needs to be cut off
                    sub_tx_mut = mut_ideal_seq[s-ideal_start:] + chr9[ideal_end:e]
                    sub_tx_wt = wt_ideal_seq[s-ideal_start:] + chr9[ideal_end:e]
                    #print(sub_tx_wt)
                    mut_transcript+=sub_tx_mut
                    wt_transcript +=sub_tx_wt
        
        if strand=='+':
            assert wt_transcript.upper()==WT_transcript_full.upper(), f'non-matching WT transcript | {val["gRNA_id"]} {val["edit"]} | {potential_splice} | {sub_tx_wt}'


            mut_seq = str(Bio.Seq.Seq(mut_transcript).transcribe().translate())

            hgvsp = hgvsp_simple(wt_seq, mut_seq)
            hgvsp_holder.append(hgvsp)
            
            potential_splice_holder.append(potential_splice)
        
        elif strand=='-':

            assert wt_transcript.reverse_complement().upper()==WT_transcript_full.upper(), f'non-matching WT transcript | {val["gRNA_id"]} {val["edit"]} | {potential_splice} | {sub_tx_wt}'


            mut_seq = str(Bio.Seq.Seq(mut_transcript).reverse_complement().transcribe().translate())

            hgvsp = hgvsp_simple(wt_seq, mut_seq)
            hgvsp_holder.append(hgvsp)
            
            potential_splice_holder.append(potential_splice)

    return hgvsp_holder, potential_splice_holder

In [9]:
def sensor_maker(guide_dataframe, chr9):
    """ 
    function for making sensors in reverse complement orientation
    42 nt sensor (10 nt on either side of protospacer + PAM sequence)
    """

    sensors_rc = []

    for i, val in guide_dataframe.iterrows():
        strand = val['strand']
        proto = val['protospacer']
        pam = val['PAM']
        proto_pam = proto+pam

        if strand == '+':
            s = val['proto_start']
            e = val['proto_end']

            sensor = str(chr9[s-11:e+10+2]).upper()

        elif strand=='-':
            s = val['proto_end']
            e = val['proto_start']

            sensor = str(Bio.Seq.Seq(chr9[s-3-10:e+10]).reverse_complement()).upper()

        assert proto_pam == sensor[10:-10], print(f'{strand} | {sensor} | {proto_pam} | {sensor[10:-10]}')

        
        sensor_rc = str(Bio.Seq.Seq(sensor).reverse_complement())

        assert len(sensor_rc)==42

        sensors_rc.append(sensor_rc)

    return sensors_rc

def sensor_alt_maker(guides, edit_outcomes):

    sensor_wt_list = []
    sensor_alt_list = []

    for i, val in edit_outcomes.iterrows():
        grna = val['gRNA_id']
        ideal_window = val['ideal_window']
        edit = val['edit']

        sensor = guides.loc[guides['gRNA_id']==grna, 'sensor_wt'].values[0]

        sensor_wt_list.append(sensor)

        sensor_rc = str(Bio.Seq.Seq(sensor).reverse_complement())
        assert ideal_window == sensor_rc[13:18]

        sensor_alt = sensor_rc[:13] + edit + sensor_rc[18:]
        sensor_alt_rc = str(Bio.Seq.Seq(sensor_alt).reverse_complement())
        sensor_alt_list.append(sensor_alt_rc)

        assert len(sensor_alt_rc)==42
        assert sensor_alt_rc != sensor
        
    edit_outcomes['sensor_wt'] = sensor_wt_list
    edit_outcomes['sensor_alt'] = sensor_alt_list

    return edit_outcomes

In [10]:
def mutation_simulator(gene, cdks, chrom_dict):

    tx = cdks.loc[cdks['Gene']==gene, 'Transcript ID'].values[0]
    protein = cdks.loc[cdks['Gene']==gene, 'Protein'].values[0]

    #--------check that it's the correct transcript and get the WT transcript sequence-------
    start_end_cds_20, start_end_cds, chrom, strand = tx_processor(tx)
    
    if chrom != 'chrX':
        chr9 = chrom_dict[int(chrom[3:])]
    else:
        chr9 = chrom_dict[chrom[3:]]
    
    transcript = ''
    for i in start_end_cds:
        s = i[0]-1
        e = i[1]
        transcript+=chr9[s:e]
    
    if strand == '+':
        WT_transcript_full = str(transcript).upper()
    elif strand =='-':
        WT_transcript_full = str(transcript.reverse_complement()).upper()

    assert str(Bio.Seq.Seq(WT_transcript_full).transcribe().translate()) == protein

    #-----------perform the mutagenesis-----------
    out = mutagenize(tx, chrom_dict)

    #-------select only hits on the CDS--------
    out = out[out['CDS_hit']==True]
    plus_df = out[out['strand']=='+']
    minus_df = out[out['strand']=='-']

    #--------and then simulate the mutations--------

    #---------plus sequence-------------
    cds_plus, CBE_edits_df, ABE_edits_df  = plus_mutate(plus_df)

    start_end_cds_20, start_end_cds, chrom, strand = tx_processor(tx)
    
    ABE1 = pd.merge(ABE_edits_df, cds_plus, on='gRNA_id')
    #return ABE1
    hgvsp_holder, potential_splice_holder = simulate_mutations_plus(ABE1, WT_transcript_full, start_end_cds,strand, chrom_dict, chrom)
    ABE1['HGVSp'] = hgvsp_holder
    ABE1['potential_splice'] = potential_splice_holder

    CBE1 = pd.merge(CBE_edits_df, cds_plus, on='gRNA_id')
    hgvsp_holder_c, potential_splice_holder_c = simulate_mutations_plus(CBE1, WT_transcript_full, start_end_cds,strand, chrom_dict, chrom)
    CBE1['HGVSp'] = hgvsp_holder_c
    CBE1['potential_splice'] = potential_splice_holder_c

    #---------minus sequence-------------
    cds_minus, CBE_edits_df, ABE_edits_df = minus_mutate(minus_df, plus_df)
    
    ABE1_minus = pd.merge(ABE_edits_df, cds_minus, on='gRNA_id')
    hgvsp_holder, potential_splice_holder = simulate_mutations_minus(ABE1_minus, WT_transcript_full, start_end_cds,strand, chrom_dict, chrom)
    ABE1_minus['HGVSp'] = hgvsp_holder
    ABE1_minus['potential_splice'] = potential_splice_holder

    CBE1_minus = pd.merge(CBE_edits_df, cds_minus, on='gRNA_id')
    hgvsp_holder_c, potential_splice_holder_c = simulate_mutations_minus(CBE1_minus, WT_transcript_full, start_end_cds, strand, chrom_dict, chrom)
    CBE1_minus['HGVSp'] = hgvsp_holder_c
    CBE1_minus['potential_splice'] = potential_splice_holder_c

    guides = pd.concat((cds_plus, cds_minus))

    if chrom != 'chrX':
        guides['chrom'] = int(chrom[3:])
    else:
        guides['chrom'] = chrom[3:]

    sensor_rc = sensor_maker(guides, chr9)
    guides['sensor_wt'] = sensor_rc
    guides['proto_G+19'] = [f'G{i[1:]}' for i in guides['protospacer']]

    cols = ['gRNA_id','protospacer', 'proto_G+19', 'chrom', 'exon', 'proto_start', 'proto_end', 'PAM', 'PAM_start',
    'PAM_end', 'ideal_start', 'ideal_end', 'ideal_window', 'strand','sensor_wt',
    'CDS_hit', 'ABE_amenable', 'CBE_amenable', ]

    guides = guides[cols]
    
    edit_outcomes = pd.concat((ABE1, CBE1, ABE1_minus, CBE1_minus)) 

    #------and get more info from the edit_outcomes---------
    complex_list = []
    mut_aa_list = []
    wt_aa_list = []
    codon_list = []
    for i, val in edit_outcomes.iterrows():
        hg = val['HGVSp']
        if '_' in hg:
            complex_list.append(True)
            mut_aa_list.append(None)
            wt_aa_list.append(None)
            codon_list.append(None)
        else:
            complex_list.append(False)
            if hg !='WT':
                mut_aa_list.append(hg[-1])
                wt_aa_list.append(hg[0])
                codon_list.append(int(hg[1:-1]))
            elif hg=='WT':
                mut_aa_list.append(None)
                wt_aa_list.append(None)
                codon_list.append(None)


    edit_outcomes['Complex'] = complex_list
    edit_outcomes['MUT_AA'] = mut_aa_list 
    edit_outcomes['WT_AA'] = wt_aa_list
    edit_outcomes['Codon'] = codon_list

    #-------determine sensor wt and alt for each-----------
    edit_outcomes = sensor_alt_maker(guides, edit_outcomes)
    

    return guides, edit_outcomes

# Generating gRNAs

In [16]:
fgfr2_3b = pd.read_excel('2025_01_14_NG_Tiling_Libraries_YMSF_FSR.xlsx')
fgfr2_3b = fgfr2_3b[['Gene', 'Transcript ID', 'Protein']]

#SELECT ONLY THE HUMAN TRANSCRIPTS FIRST
fgfr2_3b = fgfr2_3b[6:].reset_index(drop=True)
fgfr2_3b

,Gene,Transcript ID,Protein
0,Men1,ENSMUST00000113504.10,MGLKAAQKTLFPLRSIDDVVRLFAAELGREEPDLVLLSLVLGFVEH...
1,Kdm6a,ENSMUST00000044484.13,MKSCGVSLATAAAAAAAAAFGDEEKKMAAGKASGESEEASPSLTAE...
2,Kmt2a,ENSMUST00000002095.11,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPQA...
3,Kmt2b,ENSMUST00000108154.9,MAAAAGGGSCPGPGSARGRFPGRPRGSGGGGGRGGRGNGAERVRVA...
4,Kmt2c,ENSMUST00000045291.14,MSSEEDRSAEQQQPPPAPPEEPGAPAPSPAAADKRPRGRPRKDGAS...
5,Kmt2d,ENSMUST00000023741.16,MDSQKPPAEDKDSDPAADGLAAPEKPGATEPDLPILCIGEVSVPGS...


In [17]:
guide_dfs = []
edit_dfs = []
names = []

for i in list(fgfr2_3b['Gene']):
    names.append(i)
    guides, edit_outcomes = mutation_simulator(i, fgfr2_3b, chrom_dict)
    guide_dfs.append(guides)
    edit_dfs.append(edit_outcomes)
    print(f'{i} : {len(guides)} guides : {len(edit_outcomes)} edits')

Men1 : 1129 guides : 4634 edits
Kdm6a : 1903 guides : 7833 edits
Kmt2a : 6320 guides : 27371 edits
Kmt2b : 5029 guides : 21295 edits
Kmt2c : 6975 guides : 29075 edits
Kmt2d : 10243 guides : 42657 edits


In [18]:
guide_dict = dict(zip(names, guide_dfs))
edit_dict = dict(zip(names, edit_dfs))

# Generating Controls

- Intron targeting controls
- Non-targeting controls

In [19]:
from pegg import library
from pegg import base

In [20]:
#and also generate 100 non-targeting guides for prime editing
num_guides = 1000
nontarget = library.nontargeting_guides(num_guides, edit_type='base')

nontarget['proto_G+19'] = [f'G{i[2:]}' for i in nontarget['Protospacer']]

rand_seqs = np.random.choice(['A','T','C','G'], size=(1000,42))
rand_sensors_42 = ["".join(i) for i in rand_seqs]
nontarget['sensor_seq_random'] = rand_sensors_42

RE_sites_polyT = ['CGTCTC', 'GAATTC', 'GAGACG', 'TTTT']

filt = []
for i, val in nontarget.iterrows():
    p= val['Protospacer']
    s = val['sensor_seq_random']
    
    c = 0
    for k in RE_sites_polyT:
        if k in p:
            c+=1
        if k in s:
            c+=1
        else:
            continue

    if c>0:
        filt.append(True)
    else:
        filt.append(False)

#and then generating 

nontarget['polyT_or_RE_site'] = filt
nontarget = nontarget[nontarget['polyT_or_RE_site']==False].reset_index().drop(columns=['index'])
#nontarget.to_csv('non_targeting_guides.csv', index=False)

In [21]:
nontarget

,Protospacer,classification,proto_G+19,sensor_seq_random,polyT_or_RE_site
0,GACGGAGGCTAAGCGTCGCAA,non-targeting control,GCGGAGGCTAAGCGTCGCAA,AACCGCCGAAACATATGGTTCAAACGAGAAGGGAGACCAACC,False
1,GCGCTTCCGCGGCCCGTTCAA,non-targeting control,GGCTTCCGCGGCCCGTTCAA,TTGGTTCACGGGTGCTTCGCAGACAAATGTTCCACGGCCTTC,False
2,GATCGTTTCCGCTTAACGGCG,non-targeting control,GTCGTTTCCGCTTAACGGCG,CTCTTTATTGCAGAGACCCGCAAGCACCGTCAAGGCGGCACC,False
3,GGTAGGCGCGCCGCTCTCTAC,non-targeting control,GTAGGCGCGCCGCTCTCTAC,GGGTGGTAGGCTTTAGATCCAAAGAGCCGAGATGATCAGCAC,False
4,GCCATATCGGGGCGAGACATG,non-targeting control,GCATATCGGGGCGAGACATG,AAATGATAGCTAGCGATTAGCACAAAGCCTGGGGGTGGCGGC,False
...,...,...,...,...,...
820,GGACTGAAATCCAAGGACTGT,non-targeting control,GACTGAAATCCAAGGACTGT,ACTACTCAAATTTAGGGACACGATCTCATGTGGTCATTCTTA,False
821,GTAAACAAAAAGGAAATAGTT,non-targeting control,GAAACAAAAAGGAAATAGTT,CGCGGAACGGAACGTCTGGAGGCTGTAGGTTCTCTCCTCATG,False
822,GTTTCCCATGATCATTTAGTG,non-targeting control,GTTCCCATGATCATTTAGTG,TTTAAGGGATACCCGTGATGGGTGGCCATCATCCCGTGTGGT,False
823,GCTGAGTGAAAAATAAAAGTT,non-targeting control,GTGAGTGAAAAATAAAAGTT,GAACTCGCCATGTCGAAGCTCCTCGTTCAGATGTCCTTCAAT,False


# Intron targeting controls

In [22]:
tx = 'ENST00000373264.5'


def tx_processor(tx):
    cds = list(db.children(tx, order_by='+end', featuretype=['CDS']))
    start_end_cds = [[i.start, i.end] for i in cds]
    strand = db[tx].strand
    chrom = db[tx].chrom
    #print(chrom[3:])
    #print(strand)
    #print(db[tx].attributes)

    #including 20 nt buffer on either side of exon for generating PAM sequences
    start_end_cds_20 = []
    buffer = 20
    for k in start_end_cds:
        h = []
        for idx, j in enumerate(k):
            if idx==0:
                h.append(j-buffer)
            if idx==1:
                h.append(j+buffer)
        start_end_cds_20.append(h)

    return start_end_cds_20, start_end_cds, chrom, strand

In [23]:
def mutagenize_intron(tx, chrom_dict):
    
    start_end_cds_20, start_end_cds, chrom, strand = tx_processor(tx)

    intron_20 = []
    intron_lens = []
    for i, val in enumerate(start_end_cds_20):
            
        if i<len(start_end_cds_20)-1:
            intron_20.append([start_end_cds_20[i][1], start_end_cds_20[i+1][0]])

            intron_lens.append(start_end_cds_20[i+1][0] - start_end_cds_20[i][1])
            
        if i==len(start_end_cds_20)-1:
            break

    #print(intron_lens)

    intron_20_new = []
    intron_lens_new = []
    #deal with the case of very large introns that will take forever to process

    for i, val in enumerate(intron_lens):
        if val>1000:
            rand = np.random.randint(intron_20[i][0], intron_20[i][1]-1000)
            intron_20_new.append([rand, rand+1000])
            intron_lens_new.append(1000)
        else:
            intron_lens_new.append(val)
            intron_20_new.append(intron_20[i])

    start_end_cds_20 = intron_20_new


    #chr9 = chrom_dict[9]

    if chrom != 'chrX':
        chr9 = chrom_dict[int(chrom[3:])]
    else:
        chr9 = chrom_dict[chrom[3:]]
    #chr9 = chrom_dict[int(chrom[3:])]

    plus_count = 0
    minus_count = 0

    plus_pam_loc_start = []
    plus_pam_loc_end = []
    PAM_plus = []
    plus_protospacer = []
    plus_proto_start = []
    plus_proto_end = []
    plus_ideal_start = []
    plus_ideal_end = []
    plus_ideal_window = []
    plus_exon_list = []

    minus_pam_loc_start = []
    minus_pam_loc_end = []
    PAM_minus = []
    minus_protospacer = []
    minus_proto_start = []
    minus_proto_end = []
    minus_ideal_start = []
    minus_ideal_end = []
    minus_ideal_window = []
    minus_exon_list = []

    #need to modify this to run it more generally!!!
    #COME BACK TO THIS
    #exon_list = [1,2,3,4,5,6,7]
    if strand=='+':
        exon_list = [f'intron{i}_{i+1}' for i in range(1, len(start_end_cds_20)+1)]
    elif strand=='-':
        exon_list = [f'intron{i}_{i+1}' for i in range(1, len(start_end_cds_20)+1)]

    for idx1, i in enumerate(start_end_cds_20):
        s = i[0]-1
        e = i[1]

        sub_chrom=str(chr9[s:e]).upper()
        #print(sub_chrom)
        loc_list = list(range(s, e))

        for idx, k in enumerate(sub_chrom):
            #first deal with the plus strand
            if k=='G':
                
                if idx-20 >=0:
                    plus_count+=1
                    plus_pam_loc_start.append(loc_list[idx])
                    plus_pam_loc_end.append(loc_list[idx]+1)

                    PAM_plus.append(str(chr9[loc_list[idx]-1:loc_list[idx]+1]).upper())
                    
                    proto_start = loc_list[idx]-21
                    proto_end = loc_list[idx]-1
                    proto = str(chr9[proto_start: proto_end]).upper()
                    plus_protospacer.append(proto)
                    plus_proto_start.append(proto_start+1)
                    plus_proto_end.append(proto_end)

                    #and ideal editing window
                    plus_ideal_start.append(proto_start+3+1)
                    plus_ideal_end.append(proto_start+8)
                    plus_ideal_window.append(str(chr9[proto_start+3: proto_start+8]).upper())
                    plus_exon_list.append(exon_list[idx1])

            #then the minus strand
            if k=='C':

                if idx+20<=len(sub_chrom):
                    minus_count+=1

                    minus_pam_loc_end.append(loc_list[idx])
                    minus_pam_loc_start.append(loc_list[idx]+1)

                    PAM_minus.append(str(chr9[loc_list[idx]:loc_list[idx]+2].reverse_complement()).upper())

                    proto_end = loc_list[idx]+2
                    proto_start = loc_list[idx] + 22
                    proto = str(chr9[proto_end: proto_start].reverse_complement()).upper()

                    minus_protospacer.append(proto)
                    minus_proto_start.append(proto_start)
                    minus_proto_end.append(proto_end+1)

                    #and ideal editing window
                    minus_ideal_start.append(proto_start-3)
                    minus_ideal_end.append(proto_start-8+1)
                    minus_ideal_window.append(str(chr9[proto_start-8: proto_start-3].reverse_complement()).upper())
                    minus_exon_list.append(exon_list[idx1])

    col_names = ['protospacer', 'exon', 'proto_start', 'proto_end', 'PAM', 'PAM_start', 'PAM_end', 'ideal_start', 'ideal_end', 'ideal_window']

    cols = [plus_protospacer, plus_exon_list, plus_proto_start,plus_proto_end,PAM_plus,plus_pam_loc_start,plus_pam_loc_end, plus_ideal_start, plus_ideal_end, plus_ideal_window ]
    plus_df = pd.DataFrame(dict(zip(col_names, cols)))
    plus_df['strand'] = '+'


    cols_minus = [minus_protospacer, minus_exon_list, minus_proto_start,minus_proto_end,PAM_minus,minus_pam_loc_start,minus_pam_loc_end, minus_ideal_start, minus_ideal_end, minus_ideal_window ]

    minus_df = pd.DataFrame(dict(zip(col_names, cols_minus)))
    minus_df['strand'] = '-'
    
    df = pd.concat((plus_df, minus_df))

    #and determine distance to the nearest exon of each guide
    distances = []
    for k, val in df.iterrows():
        s = val['proto_start']
        e = val['proto_end']


        #make sure it doesn't fall in an exon, and compute distance to the nearest exon (minimum)

        distance_to_exon = []

        for j in start_end_cds:
            start_exon = j[0]
            end_exon = j[1]
            
            assert ((s>=start_exon) & (s<end_exon))!=True, print(val, j)
            assert ((e>=start_exon) & (e<end_exon))!=True, print(val, j)

            distances1 = [abs(s-start_exon), abs(e-end_exon), abs(e-start_exon), abs(e-end_exon)]

            distance_to_exon.append(min(distances1))

        
        distances.append(min(distance_to_exon))

    df['nearest_exon_distance'] = distances


    if chrom != 'chrX':
        df['chrom'] = int(chrom[3:])
    else:
        df['chrom'] = chrom[3:]
    

    df['proto_G+19'] = [f'G{i[1:]}' for i in df['protospacer']]

    #and generate the sensors as well
    sensor_rc = sensor_maker(df, chr9)
    df['sensor_wt'] = sensor_rc

    return df

In [24]:
intron_holder = []
gene_list = []
for i, val in fgfr2_3b.iterrows():
#for tx in list(cdks['Transcript ID']):
    tx = val['Transcript ID']
    gene = val['Gene']
    gene_list.append(gene)
    out = mutagenize_intron(tx, chrom_dict)
    out['Gene'] = gene
    intron_holder.append(out)

In [25]:
intron_dict = dict(zip(gene_list, intron_holder))
intron_dict['Men1']

,protospacer,exon,proto_start,proto_end,PAM,PAM_start,PAM_end,ideal_start,ideal_end,ideal_window,strand,nearest_exon_distance,chrom,proto_G+19,sensor_wt,Gene
0,TCAGGAGAGCAAATGAGTGT,intron1_2,6385955,6385974,TG,6385975,6385976,6385958,6385962,GGAGA,+,39,19,GCAGGAGAGCAAATGAGTGT,AGAGGTAAGACAACACTCATTTGCTCTCCTGACCACCTGTCT,Men1
1,TGTTGTCTTACCTCTTCTCA,intron1_2,6385972,6385991,TG,6385992,6385993,6385975,6385979,TGTCT,+,56,19,GGTTGTCTTACCTCTTCTCA,ATTGTTCAGACATGAGAAGAGGTAAGACAACACTCATTTGCT,Men1
2,GTCTTACCTCTTCTCATGTC,intron1_2,6385976,6385995,TG,6385996,6385997,6385979,6385983,TTACC,+,60,19,GTCTTACCTCTTCTCATGTC,CCATATTGTTCAGACATGAGAAGAGGTAAGACAACACTCATT,Men1
3,CTTCTCATGTCTGAACAATA,intron1_2,6385985,6386004,TG,6386005,6386006,6385988,6385992,CTCAT,+,69,19,GTTCTCATGTCTGAACAATA,GGCTATAAGCCATATTGTTCAGACATGAGAAGAGGTAAGACA,Men1
4,TTCTCATGTCTGAACAATAT,intron1_2,6385986,6386005,GG,6386006,6386007,6385989,6385993,TCATG,+,70,19,GTCTCATGTCTGAACAATAT,GGGCTATAAGCCATATTGTTCAGACATGAGAAGAGGTAAGAC,Men1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,GGATCATCCGTGGGACCTCT,intron8_9,6389683,6389664,AG,6389662,6389661,6389680,6389676,TCATC,-,34,19,GGATCATCCGTGGGACCTCT,GGAGCTGAGCCTAGAGGTCCCACGGATGATCCTCAGCCTTTC,Men1
534,AAGGCTGAGGATCATCCGTG,intron8_9,6389691,6389672,GG,6389670,6389669,6389688,6389684,GCTGA,-,26,19,GAGGCTGAGGATCATCCGTG,GCCTAGAGGTCCCACGGATGATCCTCAGCCTTTCACTTGGCT,Men1
535,AAAGGCTGAGGATCATCCGT,intron8_9,6389692,6389673,GG,6389671,6389670,6389689,6389685,GGCTG,-,25,19,GAAGGCTGAGGATCATCCGT,CCTAGAGGTCCCACGGATGATCCTCAGCCTTTCACTTGGCTT,Men1
536,GAAAGGCTGAGGATCATCCG,intron8_9,6389693,6389674,TG,6389672,6389671,6389690,6389686,AGGCT,-,24,19,GAAAGGCTGAGGATCATCCG,CTAGAGGTCCCACGGATGATCCTCAGCCTTTCACTTGGCTTG,Men1


In [26]:
min_exon_dist_cutoff = 30
num_guides = 2

holder = []
for i in intron_dict.keys():

    cdk9_introns = intron_dict[i]
    intron_names = np.unique(cdk9_introns['exon'])

    cdk9_introns = cdk9_introns[cdk9_introns['nearest_exon_distance']>=min_exon_dist_cutoff]

    #then pre-filter out polyT sequences and RE sites
    RE_sites_polyT = ['CGTCTC', 'GAATTC', 'GAGACG', 'TTTT']

    filt = []
    for i, val in cdk9_introns.iterrows():
        p= val['protospacer']
        s = val['sensor_wt']
        
        c = 0
        for k in RE_sites_polyT:
            if k in p:
                c+=1
            if k in s:
                c+=1
            else:
                continue

        if c>0:
            filt.append(True)
        else:
            filt.append(False)

    
    cdk9_introns['polyT_or_RE_site'] = filt
    cdk9_introns = cdk9_introns[cdk9_introns['polyT_or_RE_site']==False].reset_index()

    #and then randomly picking 4 guides per intron
    for k in intron_names:
        subset = cdk9_introns[cdk9_introns['exon']==k].reset_index()

        a = list(range(len(subset)))
        #print(min(num_guides, len(a)))
        choices = np.random.choice(a, size=min(num_guides, len(a)), replace=False)
        
        holder.append(subset.iloc[choices])

In [27]:
intron_guides_select = pd.concat(holder)
intron_guides_select = intron_guides_select.drop(columns = ['level_0', 'index'])
intron_guides_select = intron_guides_select.reset_index().drop(columns=['index'])
intron_guides_select

#and saving i
#intron_guides_select.to_csv('intron_guides_selected/intron_guides_selected.csv', index=False)
np.unique(intron_guides_select['Gene'], return_counts=True)

(array(['Kdm6a', 'Kmt2a', 'Kmt2b', 'Kmt2c', 'Kmt2d', 'Men1'], dtype=object),
 array([ 54,  69,  66, 113, 101,  16]))

In [28]:
u, c = np.unique(intron_guides_select['Gene'], return_counts=True)
dict_i = dict(zip(u, c))

for i in intron_dict.keys():

    cdk9_introns = intron_dict[i]
    intron_names = np.unique(cdk9_introns['exon'])

    print(f'{i} : {dict_i[i]}/{len(intron_names)*num_guides}')

Men1 : 16/16
Kdm6a : 54/56
Kmt2a : 69/70
Kmt2b : 66/72
Kmt2c : 113/116
Kmt2d : 101/106


# Combining, adding BCs, and generating library

- Filtering for RE sites and polyT sites as well
- And creating a new gRNA_id for each (should include the gene name)
- Add 5% non-targeting controls
    - Want these to overlap for different subpools...


In [29]:
genes = list(fgfr2_3b['Gene'])
genes

['Men1', 'Kdm6a', 'Kmt2a', 'Kmt2b', 'Kmt2c', 'Kmt2d']

In [30]:
#first combine the guide
genes = list(fgfr2_3b['Gene'])

guide_holder = []
for i in genes:
    d = guide_dict[i]
    d['Gene'] = i
    guide_holder.append(d)

cols = ['gRNA_id', 'Gene','protospacer', 'proto_G+19', 'chrom', 'exon', 'proto_start',
       'proto_end', 'PAM', 'PAM_start', 'PAM_end', 'ideal_start', 'ideal_end',
       'ideal_window', 'strand', 'sensor_wt', 'CDS_hit', 'ABE_amenable',
       'CBE_amenable']

combined_guides = pd.concat(guide_holder)[cols]
combined_guides['classification'] = 'targeting'
combined_guides

,gRNA_id,Gene,protospacer,proto_G+19,chrom,exon,proto_start,proto_end,PAM,PAM_start,PAM_end,ideal_start,ideal_end,ideal_window,strand,sensor_wt,CDS_hit,ABE_amenable,CBE_amenable,classification
0,gRNA_1,Men1,GCCGCCATGGGGCTGAAGGC,GCCGCCATGGGGCTGAAGGC,19,1,6385485,6385504,CG,6385505,6385506,6385488,6385492,GCCAT,+,GTCTTCTGGGCGGCCTTCAGCCCCATGGCGGCGGGCGGTGGC,True,True,True,targeting
1,gRNA_2,Men1,CATGGGGCTGAAGGCCGCCC,GATGGGGCTGAAGGCCGCCC,19,1,6385490,6385509,AG,6385510,6385511,6385493,6385497,GGGGC,+,ACAGCGTCTTCTGGGCGGCCTTCAGCCCCATGGCGGCGGGCG,True,False,True,targeting
2,gRNA_3,Men1,GGGGCTGAAGGCCGCCCAGA,GGGGCTGAAGGCCGCCCAGA,19,1,6385493,6385512,AG,6385513,6385514,6385496,6385500,GCTGA,+,GGAACAGCGTCTTCTGGGCGGCCTTCAGCCCCATGGCGGCGG,True,True,True,targeting
3,gRNA_4,Men1,GCTGAAGGCCGCCCAGAAGA,GCTGAAGGCCGCCCAGAAGA,19,1,6385496,6385515,CG,6385516,6385517,6385499,6385503,GAAGG,+,GAGGGAACAGCGTCTTCTGGGCGGCCTTCAGCCCCATGGCGG,True,True,False,targeting
4,gRNA_5,Men1,GAAGGCCGCCCAGAAGACGC,GAAGGCCGCCCAGAAGACGC,19,1,6385499,6385518,TG,6385519,6385520,6385502,6385506,GGCCG,+,GCAGAGGGAACAGCGTCTTCTGGGCGGCCTTCAGCCCCATGG,True,False,True,targeting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4678,gRNA_10239,Kmt2d,AGCCAGAAGCCGCCTGCTGA,GGCCAGAAGCCGCCTGCTGA,15,1,98764595,98764576,AG,98764574,98764573,98764592,98764588,CAGAA,-,GAATCTTTATCTTCAGCAGGCGGCTTCTGGCTGTCCATCCCT,True,True,True,targeting
4679,gRNA_10240,Kmt2d,GACAGCCAGAAGCCGCCTGC,GACAGCCAGAAGCCGCCTGC,15,1,98764598,98764579,TG,98764577,98764576,98764595,98764591,AGCCA,-,TCTTTATCTTCAGCAGGCGGCTTCTGGCTGTCCATCCCTCTC,True,True,True,targeting
4680,gRNA_10241,Kmt2d,ATGGACAGCCAGAAGCCGCC,GTGGACAGCCAGAAGCCGCC,15,1,98764601,98764582,TG,98764580,98764579,98764598,98764594,GACAG,-,TTATCTTCAGCAGGCGGCTTCTGGCTGTCCATCCCTCTCTCC,True,True,True,targeting
4681,gRNA_10242,Kmt2d,AGGGATGGACAGCCAGAAGC,GGGGATGGACAGCCAGAAGC,15,1,98764605,98764586,CG,98764584,98764583,98764602,98764598,GATGG,-,CTTCAGCAGGCGGCTTCTGGCTGTCCATCCCTCTCTCCGACT,True,True,False,targeting


In [31]:
genes

['Men1', 'Kdm6a', 'Kmt2a', 'Kmt2b', 'Kmt2c', 'Kmt2d']

In [32]:
#nontargeting = pd.read_csv('non_targeting_guides.csv').rename(columns = {'Protospacer':'protospacer', 'sensor_seq_random':'sensor_wt'}).drop(columns = ['polyT_or_RE_site'])
nontargeting = nontarget.rename(columns = {'Protospacer':'protospacer', 'sensor_seq_random':'sensor_wt'}).drop(columns = ['polyT_or_RE_site'])

intron_guides_select['classification'] = 'intron'

#combining exon and intron-targeting guides
targeting = pd.concat((combined_guides, intron_guides_select))

targeting.loc[targeting['Gene']==genes[0], 'Pool'] = 'F2-R1'
targeting.loc[targeting['Gene']==genes[1], 'Pool'] = 'F2-R2'
targeting.loc[targeting['Gene']==genes[2], 'Pool'] = 'F2-R3'
targeting.loc[targeting['Gene']==genes[3], 'Pool'] = 'F2-R4'
targeting.loc[targeting['Gene']==genes[4], 'Pool'] = 'F2-R5'
targeting.loc[targeting['Gene']==genes[5], 'Pool'] = 'F2-R6'

targeting = targeting.drop(columns = ['polyT_or_RE_site'])


In [33]:
#determining number of non-targeting guides to include for each
fraction_nt = .03

size1 = int(len(targeting[targeting['Pool']=='F2-R1'])*fraction_nt)
size2 = int(len(targeting[targeting['Pool']=='F2-R2'])*fraction_nt)
size3 = int(len(targeting[targeting['Pool']=='F2-R3'])*fraction_nt)
size4 = int(len(targeting[targeting['Pool']=='F2-R4'])*fraction_nt)
size5 = int(len(targeting[targeting['Pool']=='F2-R5'])*fraction_nt)
size6 = int(len(targeting[targeting['Pool']=='F2-R6'])*fraction_nt)

nt_subpool1 = nontargeting[:size1]
nt_subpool2 = nontargeting[:size2]
nt_subpool3 = nontargeting[:size3]
nt_subpool4 = nontargeting[:size4]
nt_subpool5 = nontargeting[:size5]
nt_subpool6 = nontargeting[:size6]

nt_subpool1['Pool'] = 'F2-R1'
nt_subpool2['Pool'] = 'F2-R2'
nt_subpool3['Pool'] = 'F2-R3'
nt_subpool4['Pool'] = 'F2-R4'
nt_subpool5['Pool'] = 'F2-R5'
nt_subpool6['Pool'] = 'F2-R6'

#and then adding these to the rest of the guides
combined_library = pd.concat([targeting, nt_subpool1, nt_subpool2, nt_subpool3, nt_subpool4, nt_subpool5, nt_subpool6])


In [34]:
for i in range(1,7):
    pool = f'F2-R{i}'
    subset = combined_library[combined_library['Pool']==pool]
    u, c = np.unique(subset['classification'], return_counts=True)
    print(dict(zip(u,c)))

{'intron': 16, 'non-targeting control': 34, 'targeting': 1129}
{'intron': 54, 'non-targeting control': 58, 'targeting': 1903}
{'intron': 69, 'non-targeting control': 191, 'targeting': 6320}
{'intron': 66, 'non-targeting control': 152, 'targeting': 5029}
{'intron': 113, 'non-targeting control': 212, 'targeting': 6975}
{'intron': 101, 'non-targeting control': 310, 'targeting': 10243}


In [35]:
#adding hamming barcodes
hamming = pd.read_csv('Hamming_BC_fil.csv')

#modified so there's no overlap in barcode
hamming = hamming[40000:]

#first filter out RE sites EcoRI and Esp3I
RE_sites = ['CGTCTC', 'GAATTC', 'GAGACG']

contains_RE = []
for i in list(hamming['Hammingbarcode']):
    c=0
    for k in RE_sites_polyT:
        if k in i:
            c+=1
    
    if c>0:
        contains_RE.append(True)
    elif c==0:
        contains_RE.append(False)

hamming['contains_RE'] = contains_RE

ham = hamming[hamming['contains_RE']==False]
#decreasing the size to 13 to keep oligo length at 250
ham_bc = [i[:13] for i in list(ham['Hammingbarcode'])]

combined_library['Hamming_BC'] = ham_bc[:len(combined_library)]

In [36]:
#------oligo generation tool
#----taken/modified from PEGG.base
def base_oligo_generator(peg_df, five_prime_adapter = 'AGCGTACACGTCTCACACC',three_prime_adapter = 'GAATTCTAGATCCGGTCGTCAAC',
                gRNA_scaff = 'GTTTAAGAGCTATGCTGGAAACAGCATAGCAAGTTTAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGC'):

    """
    A tool for automatically generating oligos from the output of run().
    Returns input dataframe with new columns containing gRNA oligo with or without sensor.

    Parameters
    ----------
    peg_df
        *type = pd.DataFrame*
        
        A dataframe containing the gRNAs for the selected input mutations. Generated by run_base() or gRNA_generator()

    five_prime_adapter
        *type = str*
        
        5' Prime Adapter. The automatically provided 5' adapter contains an Esp3I (BsmBI) site. Can be swapped with 
        whatever input string user wants.
    
    three_prime_adapter
        *type = str*
        
        5' Prime Adapter. The automatically provided 5' adapter contains an Esp3I (BsmBI) site. Can be swapped with 
        whatever input string user wants.
        
    gRNA_scaff
        *type = str*
        
        gRNA scaffold region. Automatically set to a functional gRNA scaffold. Can be swapped with 
        whatever input string user wants.
    
    """
        
        
    u6_term = 'TTTTTTT'

    base_oligos = []

    for i, val in peg_df.iterrows():
        proto = val['proto_G+19']
        #extension = val['RTT_PBS']
        sensor = val["sensor_wt"]
        bc = val['Hamming_BC']

        pool = val['Pool']

        f_identity = pool.split('-')[0]
        r_identity = pool.split('-')[1]

        f_dict = dict(zip(['F1','F2', 'F3', 'F4', 'F5', 'F6'],
                          ['AGGCACTTGCTCGTACGACG',
                           'GTGTAACCCGTAGGGCACCT',
                           'CAGCGCCAATGGGCTTTCGA',
                           'CTACAGGTACCGGTCCTGAG',
                           'CATGTTGCCCTGAGGCACAG',
                           'GGTCGTCGCATCACAATGCG']))
        
        r_dict = dict(zip(['R1','R2','R3','R4','R5','R6'],
                          ['TTAAGGTGCCGGGCCCACAT',
                           'GTCGAAGGACTGCTCTCGAC',
                           'CGACAGGCTCTTAAGCGGCT',
                           'CGGATCGTCACGCTAGGTAC',
                           'AGCCTTTCGGGACCTAACGG',
                           'CGTCACATTGGCGCTCGAGA']))


        F = f_dict[f_identity]
        R = r_dict[r_identity]

        gRNA_full = F + five_prime_adapter + proto + gRNA_scaff + u6_term + sensor + bc + three_prime_adapter + R
        base_oligos.append(gRNA_full)

    
    peg_df['gRNA_oligo'] = base_oligos
    
    return peg_df

In [37]:
combined_library = base_oligo_generator(combined_library)
combined_library = combined_library.reset_index().drop(columns='index')

In [38]:
#counting polyT sequence (not filtering these though)

#checking for presence of polyT sequence in proto_G+19
polyT = pd.DataFrame(dict(zip(['Gene', 'polyT_count'], [list(fgfr2_3b['Gene']), np.zeros(6)])))
for i, val in combined_library.iterrows():
    g = val['Gene']

    if 'TTTT' in val['proto_G+19']:
        polyT.loc[polyT['Gene']==g, 'polyT_count']+=1
        combined_library.loc[i, 'polyT'] = True
    else:
        combined_library.loc[i, 'polyT'] = False


polyT

,Gene,polyT_count
0,Men1,13.0
1,Kdm6a,149.0
2,Kmt2a,352.0
3,Kmt2b,101.0
4,Kmt2c,428.0
5,Kmt2d,212.0


In [39]:
#filtering for RE sites (making sure that there are only 2)

RE_sites= ['CGTCTC', 'GAATTC', 'GAGACG']
num_RE_sites = []

count = 0

for oligo in list(combined_library['gRNA_oligo']):
    total = 0
    for j in RE_sites:
        total += oligo.count(j)

    if total>2:
        count+=1
    num_RE_sites.append(total)

combined_library['RE_sites'] = num_RE_sites


In [40]:
combined_library_filtered = combined_library[combined_library['RE_sites']==2].reset_index()
combined_library_filtered = combined_library_filtered.rename(columns = {'gRNA_id':'gRNA_id_OLD'})

t2 = combined_library_filtered[combined_library_filtered['classification']=='targeting']
u, c= np.unique(t2['Gene'], return_counts=True)
print(dict(zip(u,c)))
t3 = combined_library[combined_library['classification']=='targeting']
u, c= np.unique(t3['Gene'], return_counts=True)
print(dict(zip(u,c)))

{'Kdm6a': 1873, 'Kmt2a': 6185, 'Kmt2b': 4917, 'Kmt2c': 6749, 'Kmt2d': 10089, 'Men1': 1105}
{'Kdm6a': 1903, 'Kmt2a': 6320, 'Kmt2b': 5029, 'Kmt2c': 6975, 'Kmt2d': 10243, 'Men1': 1129}


In [50]:
combined_library_filtered


type_dict = {'intron':'intron', 'non-targeting control':'nt', 'targeting':'targ'}
#adding gRNA_ids to this
ids = []
for i, val in combined_library_filtered.iterrows():
    g = val['Gene']
    class1 = val['classification']
    class2 = type_dict[class1]

    if class2 == 'nt':
        name = f'gRNA_mouse_{class2}_{i+1}'
    else:
        name = f'gRNA_mouse_{g}_{class2}_{i+1}'
    
    ids.append(name)

combined_library_filtered['gRNA_id'] = ids

In [51]:
cols = ['gRNA_id', 'Gene', 'protospacer', 'proto_G+19', 'chrom', 'exon',
       'proto_start', 'proto_end', 'PAM', 'PAM_start', 'PAM_end',
       'ideal_start', 'ideal_end', 'ideal_window', 'strand', 'sensor_wt',
       'CDS_hit', 'ABE_amenable', 'CBE_amenable', 'classification',
       'nearest_exon_distance', 'Pool', 'Hamming_BC', 'gRNA_oligo', 'polyT', 'RE_sites', 'gRNA_id_OLD',]

combined_library_filtered = combined_library_filtered[cols]

In [52]:
combined_library_filtered

,gRNA_id,Gene,protospacer,proto_G+19,chrom,exon,proto_start,proto_end,PAM,PAM_start,...,ABE_amenable,CBE_amenable,classification,nearest_exon_distance,Pool,Hamming_BC,gRNA_oligo,polyT,RE_sites,gRNA_id_OLD
0,gRNA_mouse_Men1_targ_1,Men1,GCCGCCATGGGGCTGAAGGC,GCCGCCATGGGGCTGAAGGC,19,1,6385485.0,6385504.0,CG,6385505.0,...,True,True,targeting,NaN,F2-R1,TTATACACAGCTA,GTGTAACCCGTAGGGCACCTAGCGTACACGTCTCACACCGCCGCCA...,False,2,gRNA_1
1,gRNA_mouse_Men1_targ_2,Men1,CATGGGGCTGAAGGCCGCCC,GATGGGGCTGAAGGCCGCCC,19,1,6385490.0,6385509.0,AG,6385510.0,...,False,True,targeting,NaN,F2-R1,ATGGTGCTCCTGG,GTGTAACCCGTAGGGCACCTAGCGTACACGTCTCACACCGATGGGG...,False,2,gRNA_2
2,gRNA_mouse_Men1_targ_3,Men1,GGGGCTGAAGGCCGCCCAGA,GGGGCTGAAGGCCGCCCAGA,19,1,6385493.0,6385512.0,AG,6385513.0,...,True,True,targeting,NaN,F2-R1,TACGACGAGCGAG,GTGTAACCCGTAGGGCACCTAGCGTACACGTCTCACACCGGGGCTG...,False,2,gRNA_3
3,gRNA_mouse_Men1_targ_4,Men1,GCTGAAGGCCGCCCAGAAGA,GCTGAAGGCCGCCCAGAAGA,19,1,6385496.0,6385515.0,CG,6385516.0,...,True,False,targeting,NaN,F2-R1,AGCACGGAAGTGC,GTGTAACCCGTAGGGCACCTAGCGTACACGTCTCACACCGCTGAAG...,False,2,gRNA_4
4,gRNA_mouse_Men1_targ_5,Men1,GAAGGCCGCCCAGAAGACGC,GAAGGCCGCCCAGAAGACGC,19,1,6385499.0,6385518.0,TG,6385519.0,...,False,True,targeting,NaN,F2-R1,TGGGCCCGCCCAT,GTGTAACCCGTAGGGCACCTAGCGTACACGTCTCACACCGAAGGCC...,False,2,gRNA_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32283,gRNA_mouse_nt_32284,NaN,GATTGCGCAATCCTTAGGATA,GTTGCGCAATCCTTAGGATA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,non-targeting control,NaN,F2-R6,CCTGTAATCGTTC,GTGTAACCCGTAGGGCACCTAGCGTACACGTCTCACACCGTTGCGC...,False,2,NaN
32284,gRNA_mouse_nt_32285,NaN,GGAAGCGGGACCGTGTCTCAC,GAAGCGGGACCGTGTCTCAC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,non-targeting control,NaN,F2-R6,AGAAACTTACAGC,GTGTAACCCGTAGGGCACCTAGCGTACACGTCTCACACCGAAGCGG...,False,2,NaN
32285,gRNA_mouse_nt_32286,NaN,GGTGTATGATGCTTCGACTTA,GTGTATGATGCTTCGACTTA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,non-targeting control,NaN,F2-R6,AACAAGGTAACGA,GTGTAACCCGTAGGGCACCTAGCGTACACGTCTCACACCGTGTATG...,False,2,NaN
32286,gRNA_mouse_nt_32287,NaN,GCCTACTCCCGTGTGTTATCC,GCTACTCCCGTGTGTTATCC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,non-targeting control,NaN,F2-R6,TGTGGGATATAAA,GTGTAACCCGTAGGGCACCTAGCGTACACGTCTCACACCGCTACTC...,False,2,NaN


In [53]:
#final sensor check

targ1 = combined_library_filtered[combined_library_filtered['classification']!='non-targeting control']

for i, val in targ1.iterrows():
    proto = val['proto_G+19'][1:]
    sensor = str(Bio.Seq.Seq(val['sensor_wt']).reverse_complement())

    assert proto in sensor

In [54]:
print(len(np.unique(combined_library_filtered['gRNA_oligo'])))
print(len(np.unique(combined_library_filtered['proto_G+19'])))
print(len(np.unique(combined_library_filtered['sensor_wt'])))
print(len(np.unique(combined_library_filtered['gRNA_id'])))
print(len(np.unique(combined_library_filtered['Hamming_BC'])))

print(np.unique([len(i) for i in combined_library_filtered['gRNA_oligo']]))
print(np.unique(combined_library_filtered['RE_sites']))

#note that there are duplicate sgRNAs because there are paralogs (most likely)
#need to keep this in mind in the future

32288
31464
31630
32288
32288
[250]
[2]


In [55]:
#label and save
combined_library_filtered['species'] = 'Mouse'
combined_library_filtered.to_csv('Mouse_tiling_library.csv', index=False)

# modfying edit df to exclude filtered guides

Haven't done this yet.

In [56]:
genes = list(fgfr2_3b['Gene'])
genes

['Men1', 'Kdm6a', 'Kmt2a', 'Kmt2b', 'Kmt2c', 'Kmt2d']

In [57]:

edit_holder2 = []
for gene in genes:
    targ = combined_library_filtered[(combined_library_filtered['classification']=='targeting') & (combined_library_filtered['Gene']==gene)]
    id_dict = dict(zip(targ['gRNA_id_OLD'], targ['gRNA_id']))

    e = edit_dict[gene]
    e = e[e['gRNA_id'].isin(list(targ['gRNA_id_OLD']))]
    e = e.rename(columns = {'gRNA_id':'gRNA_id_OLD'}) 

    e['gRNA_id'] = [id_dict[i] for i in list(e['gRNA_id_OLD'])]
    e['Gene'] = gene

    cols = ['gRNA_id', 'edit', 'Editor', 'protospacer', 'exon', 'proto_start',
       'proto_end', 'PAM', 'PAM_start', 'PAM_end', 'ideal_start', 'ideal_end',
       'ideal_window', 'strand', 'CDS_hit', 'ABE_amenable', 'CBE_amenable',
       'HGVSp', 'potential_splice', 'Complex', 'MUT_AA', 'WT_AA', 'Codon',
       'sensor_wt', 'sensor_alt', 'Gene','gRNA_id_OLD',]
    
    e = e[cols]

    edit_holder2.append(e)

    e.to_csv(f'predicted_edit_tables/mouse_{gene}_predicted_edits.csv', index=False)

